In [ ]:
# Data Exploration
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import qgrid

from IPython.display import display, HTML
display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""
    )
)


CASES = [
         "20210422_0930.BASECASE", "gen_BLENO.TG51", "gen_CORD56CORD54", "gen_GRAV57GRAV5T4",
         "branchB_ALBERY762", "branchB_AVOI5L71G.AVO", "branchB_BARNAL72PALUE",
         "branchB_CHIN2L71G.AVO", "branchB_DONZAL72GOLF5", "branchB_FLAMAL72MENUE", "branchB_PENL5L71PENLY" 
        ]
INPUT_CASE = CASES[0]
INPUT_FILE = INPUT_CASE + "/pfsolution_AB.csv.xz"
REL_ERR_CLIPPING = 0.1

df = pd.read_csv(INPUT_FILE, sep=";", index_col="ID", skiprows=0, nrows=None)
sorted_vlevels = [str(x) for x in sorted(df.VOLT_LEVEL.unique(), reverse=True)]  # to order traces by voltage leel
df["VOLT_LEVEL"] = df["VOLT_LEVEL"].astype(str)  # to force "discrete colors" in Plotly Express
df["ABS_ERR"] = (df["VALUE_A"] - df["VALUE_B"]).abs()
df["REL_ERR"] = df["ABS_ERR"] / df["VALUE_A"].abs().clip(lower=REL_ERR_CLIPPING)


def error_figs(df, element_type, var, sorted_vlevels):
    # Relative errors vs absolute errors
    fig_relerr_abserr = px.scatter(df, y="REL_ERR", x="ABS_ERR", color="VOLT_LEVEL",
                                   color_discrete_sequence=px.colors.qualitative.D3,
                                   category_orders={"VOLT_LEVEL": sorted_vlevels},
                                   hover_name=df.index)
    fig_relerr_abserr.update_layout(                             
        title={
            'text': f"Case: <b>{INPUT_CASE}</b> ({element_type})",
            'y':0.93, 'x':0.5,
            'xanchor': 'center', 'yanchor': 'middle'},
        title_font_family="Courier New",
        title_font_color="black",
        title_font_size=24)
    fig_relerr_abserr.update_yaxes(title_text=f"REL_ERR  ({var})")
    fig_relerr_abserr.update_xaxes(title_text=f"ABS_ERR  ({var})")
    
    # Relative errors vs actual values (from CASE A)
    fig_relerr_val = px.scatter(df, y="REL_ERR", x=df.VALUE_A.abs(), color="VOLT_LEVEL",
                                color_discrete_sequence=px.colors.qualitative.D3,
                                category_orders={"VOLT_LEVEL": sorted_vlevels},
                                hover_name=df.index)
    fig_relerr_val.update_yaxes(title_text=f"REL_ERR  ({var})")
    fig_relerr_val.update_xaxes(title_text=var)
        
    return fig_relerr_abserr, fig_relerr_val

In [ ]:
# Main table of absolute and relative errors
show_err_table = qgrid.show_grid(df)
show_err_table

In [ ]:
# Bus voltages
df_f = df.loc[(df.ELEMENT_TYPE=="bus") & (df.VAR=="v")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f, "buses", "V", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Lines
df_f = df.loc[(df.ELEMENT_TYPE=="line") & (df.VAR=="p1")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "lines", "P1", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Transformers
df_f = df.loc[(df.ELEMENT_TYPE=="xfmr") & (df.VAR=="p1")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "transformers", "P1", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Phase Shifters
df_f = df.loc[(df.ELEMENT_TYPE=="psxfmr") & (df.VAR=="p1")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "phase shifters", "P1", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Taps
df_f = df.loc[(df.ELEMENT_TYPE=="xfmr") & (df.VAR=="tap")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "transformer taps", "Tap", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Phase-shiter Taps
df_f = df.loc[(df.ELEMENT_TYPE=="psxfmr") & (df.VAR=="pstap")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "phase-shifter taps", "PSTap", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Bus injections (P)
df_f = df.loc[(df.ELEMENT_TYPE=="bus") & (df.VAR=="p")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "bus injections (P)", "P", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()

In [ ]:
# Bus injections (Q)
df_f = df.loc[(df.ELEMENT_TYPE=="bus") & (df.VAR=="q")]
fig_relerr_abserr, fig_relerr_val = error_figs(df_f,  "bus injections (Q)", "Q", sorted_vlevels)
fig_relerr_abserr.show()
fig_relerr_val.show()